<a href="https://colab.research.google.com/github/textyash20/AI-prac/blob/main/Diffutoon_jupyter_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/modelscope/DiffSynth-Studio/blob/main/examples/Diffutoon/Diffutoon.ipynb modified

%cd /content
!git clone https://github.com/Artiprocher/DiffSynth-Studio
%cd /content/DiffSynth-Studio

!pip install -q einops transformers controlnet-aux==0.0.7 sentencepiece imageio imageio-ffmpeg

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/229575 -d /content/DiffSynth-Studio/models/stable_diffusion -o aingdiffusion_v12.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt -d /content/DiffSynth-Studio/models/AnimateDiff -o mm_sd_v15_v2.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth -d /content/DiffSynth-Studio/models/ControlNet -o control_v11p_sd15_lineart.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth -d /content/DiffSynth-Studio/models/ControlNet -o control_v11f1e_sd15_tile.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth -d /content/DiffSynth-Studio/models/ControlNet -o control_v11f1p_sd15_depth.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth -d /content/DiffSynth-Studio/models/ControlNet -o control_v11p_sd15_softedge.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/DiffSynth-Studio/models/Annotators -o dpt_hybrid-midas-501f0c75.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/ControlNetHED.pth -d /content/DiffSynth-Studio/models/Annotators -o ControlNetHED.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/sk_model.pth -d /content/DiffSynth-Studio/models/Annotators -o sk_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/Annotators/resolve/main/sk_model2.pth -d /content/DiffSynth-Studio/models/Annotators -o sk_model2.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/25820?type=Model&format=PickleTensor&size=full&fp=fp16" -d /content/DiffSynth-Studio/models/textual_inversion -o verybadimagenegative_v1.3.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/Diffutoon/resolve/main/input_video.mp4 -d /content -o input_video.mp4

/content
Cloning into 'DiffSynth-Studio'...
remote: Enumerating objects: 3732, done.
remote: Counting objects: 100% (1294/1294), done.
remote: Compressing objects: 100% (446/446), done.
remote: Total 3732 (delta 1035), reused 856 (delta 848), pack-reused 2438 (from 2)
Receiving objects: 100% (3732/3732), 11.52 MiB | 15.12 MiB/s, done.
Resolving deltas: 100% (2427/2427), done.
/content/DiffSynth-Studio
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0

In [2]:
def generate_video(image_path, output_path, n_frames=30, fps=30, width=512, height=512):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Unable to load image.")
        return

    # Resize image to match video resolution
    image = cv2.resize(image, (width, height))

    # Define video codec and writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' for .avi format
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for i in range(n_frames):
        # Create a variation of the image (e.g., slight brightness change)
        variation = image + (i % 2) * 10  # Alternate brightness
        variation = np.clip(variation, 0, 255).astype(np.uint8)

        # Write frame to video
        video_writer.write(variation)

    # Release the writer
    video_writer.release()
    print(f"Video saved at {output_path}")

In [3]:
import cv2
import numpy as np

In [4]:
generate_video('/content/WhatsApp Image 2025-03-31 at 13.00.51_9cbbd9af.jpg','/content/input_video.mp4',30,30)

Video saved at /content/input_video.mp4


In [6]:
!pip install modelscope ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00


In [7]:
config_stage_2_template = {
    "models": {
        "model_list": [
            "models/stable_diffusion/aingdiffusion_v12.safetensors",
            "models/AnimateDiff/mm_sd_v15_v2.ckpt",
            "models/ControlNet/control_v11f1e_sd15_tile.pth",
            "models/ControlNet/control_v11p_sd15_lineart.pth"
        ],
        "textual_inversion_folder": "models/textual_inversion",
        "device": "cuda",
        "lora_alphas": [],
        "controlnet_units": [
            {
                "processor_id": "tile",
                "model_path": "models/ControlNet/control_v11f1e_sd15_tile.pth",
                "scale": 0.5
            },
            {
                "processor_id": "lineart",
                "model_path": "models/ControlNet/control_v11p_sd15_lineart.pth",
                "scale": 0.5
            }
        ]
    },
    "data": {
        "input_frames": {
            "video_file": "/content/input_video.mp4",
            "image_folder": None,
            "height": 1024,
            "width": 1024,
            "start_frame_id": 0,
            "end_frame_id": 30
        },
        "controlnet_frames": [
            {
                "video_file": "/content/input_video.mp4",
                "image_folder": None,
                "height": 1024,
                "width": 1024,
                "start_frame_id": 0,
                "end_frame_id": 30
            },
            {
                "video_file": "/content/input_video.mp4",
                "image_folder": None,
                "height": 1024,
                "width": 1024,
                "start_frame_id": 0,
                "end_frame_id": 30
            }
        ],
        "output_folder": "/content/output",
        "fps": 25
    },
    "pipeline": {
        "seed": 0,
        "pipeline_inputs": {
            "prompt": "best quality, perfect anime illustration, light, a girl is dancing, smile, solo",
            "negative_prompt": "verybadimagenegative_v1.3",
            "cfg_scale": 7.0,
            "clip_skip": 2,
            "denoising_strength": 1.0,
            "num_inference_steps": 10,
            "animatediff_batch_size": 16,
            "animatediff_stride": 8,
            "unet_batch_size": 1,
            "controlnet_batch_size": 1,
            "cross_frame_attention": False,
            # The following parameters will be overwritten. You don't need to modify them.
            "input_frames": [],
            "num_frames": 30,
            "width": 1536,
            "height": 1536,
            "controlnet_frames": []
        }
    }
}

In [9]:
from diffsynth import SDVideoPipelineRunner

config = config_stage_2_template.copy()
config["data"]["input_frames"] = {
    "video_file": "/content/input_video.mp4",
    "image_folder": None,
    "height": 1024,
    "width": 1024,
    "start_frame_id": 0,
    "end_frame_id": 5
}
config["data"]["controlnet_frames"] = [config["data"]["input_frames"], config["data"]["input_frames"]]
config["data"]["output_folder"] = "/content/toon_video"
config["data"]["fps"] = 25

runner = SDVideoPipelineRunner()
runner.run(config)

Loading models from: models/stable_diffusion/aingdiffusion_v12.safetensors
    model_name: sd_text_encoder model_class: SDTextEncoder
    model_name: sd_unet model_class: SDUNet
    model_name: sd_vae_decoder model_class: SDVAEDecoder
    model_name: sd_vae_encoder model_class: SDVAEEncoder
    The following models are loaded: ['sd_text_encoder', 'sd_unet', 'sd_vae_decoder', 'sd_vae_encoder'].
Loading models from: models/AnimateDiff/mm_sd_v15_v2.ckpt
    model_name: sd_motion_modules model_class: SDMotionModel
    The following models are loaded: ['sd_motion_modules'].
Loading models from: models/ControlNet/control_v11f1e_sd15_tile.pth
    model_name: sd_controlnet model_class: SDControlNet
    The following models are loaded: ['sd_controlnet'].
Loading models from: models/ControlNet/control_v11p_sd15_lineart.pth
    model_name: sd_controlnet model_class: SDControlNet
    The following models are loaded: ['sd_controlnet'].
Using sd_text_encoder from models/stable_diffusion/aingdiffusio

Saving video: 100%|██████████| 5/5 [00:00<00:00,  9.29it/s]


In [10]:
import moviepy.editor
moviepy.editor.ipython_display("/content/toon_video/video.mp4")

  if event.key is 'enter':



In [13]:
from diffusers import StableDiffusionPipeline, ControlNetModel, MultiControlNetModel
import torch

# Paths to the main model and ControlNets
model_path = "/content/DiffSynth-Studio/models/stable_diffusion/aingdiffusion_v12.safetensors"
controlnet_tile_path = "/content/DiffSynth-Studio/models/ControlNet/control_v11f1e_sd15_tile.pth"
controlnet_lineart_path = "/content/DiffSynth-Studio/models/ControlNet/control_v11p_sd15_lineart.pth"

# Load ControlNet models
controlnet_tile = ControlNetModel.from_single_file(controlnet_tile_path, torch_dtype=torch.float16)
controlnet_lineart = ControlNetModel.from_single_file(controlnet_lineart_path, torch_dtype=torch.float16)
negative_embedding_path = "/content/DiffSynth-Studio/models/embeddings/verybadimagenegative_v1.3.safetensors"  # Update path

# Combine both ControlNets
multi_controlnet = MultiControlNetModel([controlnet_tile, controlnet_lineart])

# Load the Stable Diffusion pipeline with ControlNet
pipe = StableDiffusionPipeline.from_single_file(
    model_path,
    controlnet=multi_controlnet,
    torch_dtype=torch.float16
)

# Move to GPU
pipe.to("cuda")

# Run inference (you need to provide ControlNet conditioning images)
prompt = "A beautiful Ghibli-style village with cherry blossoms"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Save or display the image
image.show()
image.save("output.png")


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

Fetching 11 files:   0%|          | 0/11 [00:01<?, ?it/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Some weights of the model checkpoint were not used when initializing CLIPTextModel: 
 ['text_model.embeddings.position_ids']
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/50 [00:00<?, ?it/s]

In [15]:
!pip install controlnet-aux

In [17]:
import controlnet_aux

In [ ]:
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel
from controlnet_aux import LineartDetector  # Remove TileDetector
import torch
from PIL import Image
import numpy as np

# Paths to models
model_path = "/content/DiffSynth-Studio/models/stable_diffusion/aingdiffusion_v12.safetensors"
controlnet_tile_path = "/content/DiffSynth-Studio/models/ControlNet/control_v11f1e_sd15_tile.pth"
controlnet_lineart_path = "/content/DiffSynth-Studio/models/ControlNet/control_v11p_sd15_lineart.pth"
negative_embedding_path = "/content/DiffSynth-Studio/models/embeddings/verybadimagenegative_v1.3.safetensors"

# Load both ControlNet models
controlnet_tile = ControlNetModel.from_single_file(controlnet_tile_path, torch_dtype=torch.float16)
controlnet_lineart = ControlNetModel.from_single_file(controlnet_lineart_path, torch_dtype=torch.float16)

# Combine both ControlNet models
controlnet_models = [controlnet_tile, controlnet_lineart]

# Load the Image-to-Image Stable Diffusion Pipeline with both ControlNets
pipe = StableDiffusionControlNetImg2ImgPipeline.from_single_file(
    model_path, controlnet=controlnet_models, torch_dtype=torch.float16
)

# Move pipeline to GPU
pipe.to("cuda")

# Load Negative Embedding (Textual Inversion)
pipe.load_textual_inversion(negative_embedding_path)

# Load input image
input_image_path = "input.jpg"  # Your input image file
image = Image.open(input_image_path).convert("RGB")

# Function to generate ControlNet input images
def preprocess_for_controlnet(img, mode):
    img = np.array(img)

    if mode == "lineart":
        detector = LineartDetector()
        return detector(img)

    elif mode == "tile":
        # Tile controlnet works best with a blurred, downsampled version of the image
        tile_img = img.copy()
        tile_img = Image.fromarray(tile_img).resize((img.shape[1] // 2, img.shape[0] // 2)).resize((img.shape[1], img.shape[0]))
        return tile_img

    else:
        raise ValueError("Unsupported ControlNet mode. Use 'lineart' or 'tile'.")

# Generate both ControlNet images
controlnet_image_tile = preprocess_for_controlnet(image, mode="tile")
controlnet_image_lineart = preprocess_for_controlnet(image, mode="lineart")

# Convert both images into a format usable by the pipeline
controlnet_images = [controlnet_image_tile, controlnet_image_lineart]

# Define prompts (Ghibli art style transformation)
prompt = "Studio Ghibli style, beautiful anime landscape, warm colors, dreamy, highly detailed"
negative_prompt = "verybadimagenegative_v1.3, blurry, deformed, bad quality"

# Generate image-to-image output using both ControlNets
output = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image,  # Original image
    control_image=controlnet_images,  # List of both ControlNet images
    num_inference_steps=10,
    strength=1.0,  # Full transformation into Ghibli style
    guidance_scale=7.0,
    width=1536,
    height=1536
).images[0]

# Save and show the output
output.show()
output.save("output_ghibli.png")
